In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from src.models.net import Net
from src.models.onboard_dataset import OnboardDataset

In [4]:
# Set variables to train the model.
no_cuda = True
seed = 1

use_cuda = not no_cuda and torch.cuda.is_available()
torch.manual_seed(seed)
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [5]:
#Prepare Data Loader for Training and Validation
batch_size = 64
test_batch_size = 100

mnist_transform = transforms.Compose([
    transforms.ToTensor(), 
    transforms.Normalize((0.1307,), (0.3081,))
])

train_loader = torch.utils.data.DataLoader(
        OnboardDataset(path='./data/processed/train_set.npz', transform=mnist_transform),
        batch_size = batch_size, shuffle=True, **kwargs)

valid_loader = torch.utils.data.DataLoader(
        OnboardDataset(path='./data/processed/valid_set.npz', transform=mnist_transform),
        batch_size=test_batch_size, shuffle=True, **kwargs)

In [6]:
lr = 0.01
momentum = 0.5
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
cross_entropy_loss = nn.CrossEntropyLoss()

In [7]:

#Define Train function and Test function to validate.
def train(log_interval, model, device, train_loader, optimizer, epoch):

    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = cross_entropy_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def valid(log_interval, model, device, test_loader):
    model.eval()
    val_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in valid_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            val_loss += cross_entropy_loss(output, target).item() 
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    val_loss /= len(valid_loader.dataset)

    print('\nValid set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format
          (val_loss, correct, len(valid_loader.dataset),
        100. * correct / len(valid_loader.dataset)))

In [8]:
log_interval = 200
epoch = 1
PATH = "./models/model.pt"
train(log_interval, model, device, train_loader, optimizer, epoch)
valid(log_interval, model, device, valid_loader)
torch.save(model, PATH)

Train Epoch: 1 [0/50000 (0%)]	Loss: 2.304849
Train Epoch: 1 [12800/50000 (26%)]	Loss: 0.519648
Train Epoch: 1 [25600/50000 (51%)]	Loss: 0.251751
Train Epoch: 1 [38400/50000 (77%)]	Loss: 0.269359

Valid set: Average loss: 0.0028, Accuracy: 9188/10000 (92%)



In [9]:
model = torch.load(PATH)
model.eval()

Net(
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)